In [66]:
import pandas as pd
import csv
import numpy as np
from scorer.main import evaluate
import logging

In [236]:
def get_features(results, iclaim, vclaim, start_index):
    curr_index = start_index
    
    while curr_index < len(results):
        
        if iclaim != results.loc[curr_index, 'iclaim_id'] or vclaim == results.loc[curr_index, 'vclaim_id']:
            break
            
        curr_index += 1
        
    if curr_index == len(results) or iclaim != results.loc[curr_index, 'iclaim_id']:
        return(0, 1/cnt_main_vclaims)
    else:
        return (results.loc[curr_index, 'score'], 1/ results.loc[curr_index,'rank'])

In [70]:
def get_related_vclaims(results, iclaim):
    vclaims = []
    for index in range(len(results)):
        if results.loc[index, 'iclaim_id'] == iclaim:
            vclaims.append(results.loc[index,'vclaim_id'])
    
    return vclaims

In [217]:
def generate_ranksvm_file(main_results, secondary_results, original_results):

    qid = 1
    main_res_index = 0
    print(main_res_index)
    sec_res_index = 0
    counter_N = 0
    ranksvm_file = []
    
    while main_res_index < len(main_results):

        iclaim_id = main_results.loc[main_res_index, 'iclaim_id']    

        if counter_N < N:

            vclaim_id = main_results.loc[main_res_index, 'vclaim_id'] 

            target = 2
            if original_results is not None and vclaim_id in get_related_vclaims(original_results, iclaim_id):
                target = 1

            line = str(target) + ' qid:' + str(qid)  \
                                + ' 1:' + str(main_results.loc[main_res_index, 'score']) \
                                + ' 2:' + str(1 / main_results.loc[main_res_index, 'rank'])

            features = []
            feature_index = 3
            for res in secondary_results:

                features = get_features(res, 
                                        iclaim_id, 
                                        vclaim_id, 
                                        sec_res_index)

                for feature in features:
                    line += ' ' + str(feature_index) + ':' + str(feature)
                    feature_index += 1

            ranksvm_file.append(line)

            main_res_index += 1
            counter_N += 1

        else:
            print(qid)
            qid = qid + 1
            counter_N = 0
            main_res_index = main_res_index - N + cnt_main_vclaims
            sec_res_index += cnt_secondary_vclaims

    return ranksvm_file

In [184]:
def write_file(results, file_name):
    with open(file_name, 'w') as f:
        for line in results:
            f.write("%s\n" % line)


In [73]:
def generate_results_from_ranksvm_file(main_results, predictions_file):
    
    with open(predictions_file, 'r') as f:
        predictions = f.read().split('\n')
    
    predictions = np.array(predictions[:-1]).reshape(cnt_iclaims, N)
    
    iclaim_index = 0
    index = 0
    all_scores = {}
    
    while index < len(main_results):

        iclaim_id = main_results.loc[index, 'iclaim_id']
        scores = {}
        i = 0

        while i < N:
            scores[main_results.loc[index, 'vclaim_id']] = -float(predictions[iclaim_index][i]) 
            i = i + 1
            index = index + 1

        scores = sorted(list(scores.items()), key=lambda kv: kv[1], reverse=True)
        all_scores[iclaim_id] = scores
        iclaim_index = iclaim_index + 1
        index = index - N + cnt_main_vclaims
    
    return all_scores

In [74]:
def format_scores(scores):
    output_string = ''
    i = 1
    for iclaim_id in scores:
        i = i + 1
        for i, (vclaim_id, score) in enumerate(scores[iclaim_id]):
            output_string += f"{iclaim_id}\tQ0\t{vclaim_id}\t{i + 1}\t{score}\tDIPS\n"
    return output_string

In [75]:
def evaluate_results(test_file_path, results_file):
    maps, mrr, precisions = evaluate(test_file_path, results_file)
    logging.info(f'All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {maps}')
    logging.info(f'MRR score {mrr}')
    logging.info(f'All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {precisions}')

In [91]:
def predict(all_scores, results_file, test_file_path):
    formatted_scores = format_scores(all_scores)
    
    with open(results_file, 'w') as f:
        f.write(formatted_scores)  
    
    evaluate_results(test_file_path, results_file)

# 2a

### TRAIN

In [120]:
N = 20

In [121]:
train_results_dir = 'results/2b/train/'
es_all_file_path_train = train_results_dir + 'subtask_2b_elastic_baseline_english_text_title_vclaim.tsv'
es_text_file_path_train = train_results_dir + 'subtask_2b_elastic_baseline_english_text.tsv'
es_vclaim_file_path_train = train_results_dir + 'subtask_2b_elastic_baseline_english_vclaim.tsv'
sbert_file_path_train = train_results_dir + 'subtask_2b_sbert_english.tsv'
train_file_path = '../data/subtask-2b--english/v1/train.tsv'

In [122]:
es_all_train_results = pd.read_csv(es_all_file_path_train, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_text_train_results = pd.read_csv(es_text_file_path_train, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_vclaim_train_results = pd.read_csv(es_vclaim_file_path_train, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
sbert_train_results = pd.read_csv(sbert_file_path_train, sep = '\t', names =['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])

In [123]:
train_results = pd.read_csv(train_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'relevance'])

In [124]:
main_results = sbert_train_results
secondary_results = [es_all_train_results, es_text_train_results, es_vclaim_train_results]
original_results = train_results

In [129]:
cnt_main_vclaims = len(main_results.vclaim_id.unique())
cnt_iclaims = len(main_results.iclaim_id.unique())
cnt_secondary_vclaims = 10000

In [130]:
cnt_main_vclaims

19250

In [131]:
cnt_iclaims

472

In [132]:
cnt_other_vclaims

10000

In [133]:
ranksvm_file = generate_ranksvm_file(main_results, secondary_results, original_results)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [134]:
ranksvm_file

['1 qid:1 1:0.9999313354492188 2:1.0 3:64.81381 4:1.0 5:43.50581 6:1.0 7:64.812386 8:1.0',
 '2 qid:1 1:0.996310293674469 2:0.5 3:34.56551 4:0.5 5:34.55857 6:0.5 7:9.324701 8:0.0014792899408284023',
 '2 qid:1 1:0.9955224990844728 2:0.3333333333333333 3:31.641457 4:0.2 5:31.634447 6:0.2 7:8.51667 8:0.0010822510822510823',
 '2 qid:1 1:0.9942492246627808 2:0.25 3:17.315348 4:0.009259259259259259 5:17.311907 6:0.013333333333333334 7:13.408295 8:0.0070921985815602835',
 '2 qid:1 1:0.9938284754753112 2:0.2 3:21.21103 4:0.02857142857142857 5:21.205837 6:0.03333333333333333 7:10.933489 8:0.002544529262086514',
 '2 qid:1 1:0.9936962723731996 2:0.16666666666666666 3:28.20604 4:0.1111111111111111 5:28.20087 6:0.1111111111111111 7:10.756363 8:0.0023752969121140144',
 '2 qid:1 1:0.9909634590148926 2:0.14285714285714285 3:32.408363 4:0.3333333333333333 5:32.401863 6:0.3333333333333333 7:11.746351 8:0.0033112582781456954',
 '2 qid:1 1:0.9908628463745116 2:0.125 3:27.224806 4:0.07692307692307693 5:27.2

In [135]:
ranksvm_file_name = 'topN/2b/train/top' + str(N)
write_file(ranksvm_file, ranksvm_file_name)

1. Train on rankSVM:
   * svm_rank_learn -c 3 topN/2b/train/top20 model/2b/train/model_top20
2. Classify:
   * svm_rank_classify topN/2b/train/top20 model/2b/train/model_top20 predictions/2b/train/predictions_top20


In [136]:
rank_svm_predictions = 'predictions/2b/train/predictions_top20'
results = generate_results_from_ranksvm_file(main_results, rank_svm_predictions)

In [137]:
results

{'20121023_third_presidential_debate_0959': [('vclaim-pol-10105', 10.85587548),
  ('vclaim-pol-07194', 5.23733506),
  ('vclaim-pol-06806', 4.8361437),
  ('vclaim-pol-04113', 4.67531105),
  ('vclaim-pol-11362', 4.60763125),
  ('vclaim-pol-11538', 4.35349223),
  ('vclaim-pol-18907', 4.33093508),
  ('vclaim-pol-10425', 4.27910021),
  ('vclaim-pol-05428', 4.26104061),
  ('vclaim-pol-10659', 4.02277855),
  ('vclaim-pol-11722', 3.90276823),
  ('vclaim-pol-14383', 3.71677244),
  ('vclaim-pol-14877', 3.57311654),
  ('vclaim-pol-11407', 3.49019487),
  ('vclaim-pol-05314', 3.44491065),
  ('vclaim-pol-05300', 3.15847636),
  ('vclaim-pol-10936', 3.0624926),
  ('vclaim-pol-17467', 2.95143578),
  ('vclaim-pol-18133', 2.7209347),
  ('vclaim-pol-12582', 1.92179189)],
 '20121023_third_presidential_debate_0985': [('vclaim-pol-10507', 5.21649636),
  ('vclaim-pol-11728', 5.14131278),
  ('vclaim-pol-11204', 4.86353437),
  ('vclaim-pol-10860', 4.82610341),
  ('vclaim-pol-14679', 4.27375094),
  ('vclaim-pol-

In [138]:
rank_svm_file = 'reranked_results/2b/train/rank_top' + str(N) + '.tsv'

In [139]:
predict(results, rank_svm_file, train_file_path)

INFO : All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.5244526836158192, 0.5973104990583804, 0.6060454919962336, 0.6133901247645951, 0.6148507933333146, 0.6148507933333146, 0.6148507933333146]
INFO : MRR score 0.6329383657773489
INFO : All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.5550847457627118, 0.24223163841807904, 0.1542372881355932, 0.08283898305084746, 0.0423728813559322, 0.01694915254237288, 0.0008474576271186444]


### DEV

In [140]:
dev_results_dir = 'results/2b/dev/'
es_all_file_path_dev = dev_results_dir + 'subtask_2b_elastic_baseline_english_text_title_vclaim.tsv'
es_text_file_path_dev = dev_results_dir + 'subtask_2b_elastic_baseline_english_text.tsv'
es_vclaim_file_path_dev = dev_results_dir + 'subtask_2b_elastic_baseline_english_vclaim.tsv'
sbert_file_path_dev = dev_results_dir + 'subtask_2b_sbert_english.tsv'
dev_file_path = '../data/subtask-2b--english/v1/dev.tsv'

In [ ]:
es_all_dev_results = pd.read_csv(es_all_file_path_dev, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_text_dev_results = pd.read_csv(es_text_file_path_dev, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_vclaim_dev_results = pd.read_csv(es_vclaim_file_path_dev, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
sbert_dev_results = pd.read_csv(sbert_file_path_dev, sep = '\t', names =['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])

In [ ]:
dev_results = pd.read_csv(dev_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'relevance'])

In [ ]:
main_results = sbert_dev_results
secondary_results = [es_all_dev_results, es_text_dev_results, es_vclaim_dev_results]
original_results = dev_results

In [ ]:
cnt_main_vclaims = len(main_results.vclaim_id.unique())
cnt_iclaims = len(main_results.iclaim_id.unique())
cnt_secondary_vclaims = 10000

In [ ]:
cnt_iclaims

In [ ]:
ranksvm_file = generate_ranksvm_file(main_results, secondary_results, original_results)

In [ ]:
ranksvm_file_name = 'topN/2b/dev/top' + str(N)
write_file(ranksvm_file, ranksvm_file_name)

Classify:
    * svm_rank_classify topN/2b/dev/top20 model/2b/train/model_top20 predictions/2b/dev/predictions_top20


In [151]:
rank_svm_predictions = 'predictions/2b/dev/predictions_top20'
results = generate_results_from_ranksvm_file(main_results, rank_svm_predictions)

In [152]:
results

{'20170822_Trump_phoenix_0204': [('vclaim-pol-02047', 3.39267339),
  ('vclaim-pol-03145', 2.97485301),
  ('vclaim-pol-02450', 2.75832777),
  ('vclaim-pol-02872', 2.45762723),
  ('vclaim-pol-07110', 2.43032424),
  ('vclaim-pol-06771', 2.40296712),
  ('vclaim-pol-05453', 2.33663454),
  ('vclaim-pol-05839', 2.0129774),
  ('vclaim-pol-18502', 2.01100866),
  ('vclaim-pol-07551', 1.89987648),
  ('vclaim-pol-03907', 1.80130483),
  ('vclaim-pol-15315', 1.75578199),
  ('vclaim-pol-18963', 1.69750971),
  ('vclaim-pol-02341', 1.6724),
  ('vclaim-pol-17942', 1.63554621),
  ('vclaim-pol-06710', 1.48989417),
  ('vclaim-pol-02524', 1.32819897),
  ('vclaim-pol-02185', 1.28980569),
  ('vclaim-pol-12884', 1.26971012),
  ('vclaim-pol-05831', 1.02923286)],
 '20170822_Trump_phoenix_0259': [('vclaim-pol-02994', 7.08410001),
  ('vclaim-pol-04215', 6.24369384),
  ('vclaim-pol-02328', 5.80402144),
  ('vclaim-pol-05474', 5.66825666),
  ('vclaim-pol-16936', 5.30824354),
  ('vclaim-pol-13135', 4.87547841),
  ('vc

In [153]:
rank_svm_file = 'reranked_results/2b/dev/rank_top' + str(N) + '.tsv'

In [154]:
predict(results, rank_svm_file, dev_file_path)

INFO : All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.4019607843137255, 0.4647525676937442, 0.4704248366013072, 0.48368680805655595, 0.4871632521101132, 0.4871632521101132, 0.4871632521101132]
INFO : MRR score 0.5007154911875623
INFO : All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.42857142857142855, 0.1904761904761904, 0.1210084033613445, 0.07058823529411762, 0.03823529411764704, 0.015294117647058824, 0.0007647058823529414]


### TEST

In [237]:
test_results_dir = 'results/2b/test/'
es_all_file_path_test = test_results_dir + 'subtask_2b_elastic_baseline_english_test_text_title_vclaim.tsv'
es_text_file_path_test = test_results_dir + 'subtask_2b_elastic_baseline_english_test_text.tsv'
es_vclaim_file_path_test = test_results_dir + 'subtask_2b_elastic_baseline_english_test_vclaim.tsv'
sbert_file_path_test = test_results_dir + 'subtask_2b_sbert_english_test.tsv'
test_file_path = '../data/subtask-2b--english/task2b-test.tsv'

In [238]:
es_all_test_results = pd.read_csv(es_all_file_path_test, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_text_test_results = pd.read_csv(es_text_file_path_test, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_vclaim_test_results = pd.read_csv(es_vclaim_file_path_test, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
sbert_test_results = pd.read_csv(sbert_file_path_test, sep = '\t', names =['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])

In [239]:
test_results = pd.read_csv(test_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'relevance'])

In [158]:
N = 20

In [159]:
main_results = sbert_test_results
other_results = [es_all_test_results, es_text_test_results, es_vclaim_test_results]
original_results = test_results

In [160]:
cnt_main_vclaims = len(main_results.vclaim_id.unique())
cnt_iclaims = len(main_results.iclaim_id.unique())
cnt_other_vclaims = 10000

In [161]:
ranksvm_file = generate_ranksvm_file(main_results, other_results, None)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [162]:
ranksvm_file

['2 qid:1 1:0.9801721572875975 2:1.0 3:15.449619 4:0.03125 5:15.440286 6:0.041666666666666664 7:2.8547435 8:0.00014076576576576576',
 '2 qid:1 1:0.97994464635849 2:0.5 3:7.3533015 4:0.0004161464835622139 5:7.354923 6:0.000687757909215956 7:0 8:5.194805194805195e-05',
 '2 qid:1 1:0.9722243547439576 2:0.3333333333333333 3:19.359377 4:0.14285714285714285 5:19.35568 6:0.2 7:7.969735 8:0.0020242914979757085',
 '2 qid:1 1:0.9676984548568726 2:0.25 3:8.394648 4:0.0007288629737609329 5:8.39695 6:0.001152073732718894 7:4.843923 8:0.0003405994550408719',
 '2 qid:1 1:0.9656224250793456 2:0.2 3:6.250973 4:0.0002509410288582183 5:4.1932025 6:0.00018034265103697024 7:6.246932 8:0.0007204610951008645',
 '2 qid:1 1:0.9591515064239502 2:0.16666666666666666 3:0 4:5.194805194805195e-05 5:3.5504305 6:0.0001410636196924813 7:0 8:5.194805194805195e-05',
 '2 qid:1 1:0.9580981731414796 2:0.14285714285714285 3:17.124844 4:0.06666666666666667 5:16.693083 6:0.09090909090909091 7:17.079506 8:0.2',
 '2 qid:1 1:0.9

In [163]:
write_file(ranksvm_file, 'topN/2b/test/top' + str(N))

 Classify:
   * svm_rank_classify topN/2b/test/top20 model/2b/train/model_top20 predictions/2b/test/predictions_top20


In [164]:
rank_svm_predictions = 'predictions/2b/test/predictions_top20'
results = generate_results_from_ranksvm_file(main_results, rank_svm_predictions)

In [165]:
print(results)

{'2019_oct_03_221': [('vclaim-pol-01412', 3.49658754), ('vclaim-pol-01506', 3.23003578), ('vclaim-pol-04098', 3.17960714), ('vclaim-pol-17334', 3.16023582), ('vclaim-pol-04675', 2.91715853), ('vclaim-pol-01425', 2.86110279), ('vclaim-pol-01659', 1.94099513), ('vclaim-pol-01436', 1.63973573), ('vclaim-pol-17437', 1.57227424), ('vclaim-pol-17457', 1.5003882), ('vclaim-pol-14096', 1.46788117), ('vclaim-pol-00467', 1.35410952), ('vclaim-pol-18089', 1.34394088), ('vclaim-pol-15570', 1.22859861), ('vclaim-pol-05484', 1.19893559), ('vclaim-pol-01460', 1.11096719), ('vclaim-pol-00375', 1.10061851), ('vclaim-pol-03263', 1.09695807), ('vclaim-pol-01325', 0.85474838), ('vclaim-pol-15646', 0.49070587)], '2019_nov_21_235': [('vclaim-pol-16814', 8.14737463), ('vclaim-pol-17456', 7.40531399), ('vclaim-pol-13172', 4.40812268), ('vclaim-pol-18761', 4.25637285), ('vclaim-pol-15503', 4.23356108), ('vclaim-pol-11055', 4.22832069), ('vclaim-pol-15718', 4.00805409), ('vclaim-pol-11725', 3.91932663), ('vclai

In [166]:
rank_svm_file = 'reranked_results/2b/test/rank_top' + str(N) + '.tsv'

In [167]:
predict(results, rank_svm_file, test_file_path)

INFO : All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.3987341772151899, 0.44936708860759494, 0.4533755274261603, 0.45443037974683553, 0.4558782162654091, 0.4558782162654091, 0.4558782162654091]
INFO : MRR score 0.4526433358153388
INFO : All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.4050632911392405, 0.189873417721519, 0.12151898734177215, 0.06202531645569619, 0.03227848101265822, 0.012911392405063294, 0.0006455696202531646]


# 2a

### Test

In [220]:
test_results_dir = 'results/2a/test/'
es_all_file_path_test = test_results_dir + 'subtask_2a_elastic_baseline_english_vclaim_title_subtitle_date.tsv'
es_vt_file_path_test = test_results_dir + 'subtask_2a_elastic_baseline_english_vclaim_title.tsv'
es_vclaim_file_path_test = test_results_dir + 'subtask_2a_elastic_baseline_english_vclaim.tsv'
sbert_file_path_test = test_results_dir + 'subtask_2a_sbert_english.tsv'
test_file_path = '../data/subtask-2a--english/qrels-test.tsv'

In [221]:
es_all_test_results = pd.read_csv(es_all_file_path_test, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_vt_test_results = pd.read_csv(es_vt_file_path_test, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
es_vclaim_test_results = pd.read_csv(es_vclaim_file_path_test, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])
sbert_test_results = pd.read_csv(sbert_file_path_test, sep = '\t', names =['iclaim_id', 'Q0', 'vclaim_id', 'rank', 'score', 'method'])

In [222]:
test_results = pd.read_csv(test_file_path, sep = '\t', names = ['iclaim_id', 'Q0', 'vclaim_id', 'relevance'])

In [223]:
N = 20

In [224]:
main_results = sbert_test_results
other_results = [es_all_test_results, es_vt_test_results, es_vclaim_test_results]
original_results = test_results

In [225]:
cnt_main_vclaims = len(main_results.vclaim_id.unique())
cnt_iclaims = len(main_results.iclaim_id.unique())
cnt_other_vclaims = 10000

In [226]:
print(cnt_main_vclaims, cnt_iclaims)

13824 202


In [227]:
ranksvm_file = generate_ranksvm_file(main_results, other_results, None)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202


In [228]:
ranksvm_file

['2 qid:1 1:0.9938815832138062 2:1.0 3:24.844534 4:0.14285714285714285 5:24.834959 6:0.14285714285714285 7:24.844404 8:0.2',
 '2 qid:1 1:0.9739006161689758 2:0.5 3:2.769501 4:0.00016160310277957336 5:2.7692096 6:0.00026602819898909286 7:2.2991285 8:0.0002627430373095113',
 '2 qid:1 1:0.9706859588623048 2:0.3333333333333333 3:25.395958 4:0.16666666666666666 5:25.38294 6:0.16666666666666666 7:25.395786 8:0.25',
 '2 qid:1 1:0.9694289565086364 2:0.25 3:2.9958174 4:0.000177999288002848 5:1.6894039 6:0.0001313197636244255 7:1.0907807 8:0.00012868356710848025',
 '2 qid:1 1:0.9682180285453796 2:0.2 3:28.89161 4:0.3333333333333333 5:28.883038 6:0.3333333333333333 7:27.070944 8:0.3333333333333333',
 '2 qid:1 1:0.963232398033142 2:0.16666666666666666 3:3.2578883 4:0.00020408163265306123 5:2.0980146 6:0.0001762114537444934 7:1.87219 8:0.0001866019779809666',
 '2 qid:1 1:0.9630980491638184 2:0.14285714285714285 3:2.9958174 4:0.00018162005085361425 5:2.0165534 6:0.00016488046166529267 7:1.4727571 8:

In [229]:
write_file(ranksvm_file, 'topN/2a/test/top' + str(N))

 Classify:
   * svm_rank_classify topN/2a/test/top20 model/2a/train/model_top20 predictions/2a/test/predictions_top20


In [230]:
rank_svm_predictions = 'predictions/2a/test/predictions_top20'
results = generate_results_from_ranksvm_file(main_results, rank_svm_predictions)

In [231]:
print(results)

{'tweet-sno-1200': [('vclaim-sno-black-raspberry-chocolate-chip-oreos', 4.99380618), ('vclaim-sno-oreos-fake-tweet', 4.43303295), ('vclaim-sno-lady-gaga-oreo', 4.25527496), ('vclaim-sno-secret-symbols-oreo-cookies', 3.80887285), ('vclaim-sno-oreo-moving-mexico', 2.0763942), ('vclaim-sno-lol-surprise-dolls-water', 1.40250436), ('vclaim-sno-biden-tweet-pandemic', 1.23238991), ('vclaim-sno-red-velvet-oreos', 1.08217766), ('vclaim-sno-capitol-invasion-lego-set', 0.79921347), ('vclaim-sno-kaleb-shriners-hospitals-death', 0.77130711), ('vclaim-sno-truth-costco-kirkland-meat', 0.59232487), ('vclaim-sno-disney-pixar-boo', 0.58930353), ('vclaim-sno-free-coke-bag-scam', 0.517075), ('vclaim-sno-twitter-cammo-trump-covid', 0.4325913), ('vclaim-sno-did-mars-release-a-pod-product-in-the-wake-of-the-tide-pod-challenge-controversy', 0.38700615), ('vclaim-sno-claim-bugs-found-in-ferrero-rocher-candy', 0.20677588), ('vclaim-sno-costco-lying-about-cage-free-eggs', 0.18749038), ('vclaim-sno-monkey-busines

In [232]:
rank_svm_file = 'reranked_results/2a/test/rank_top' + str(N) + '.tsv'

In [233]:
predict(results, rank_svm_file, test_file_path)

INFO : All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.8217821782178217, 0.8638613861386139, 0.8688118811881188, 0.8706192047776204, 0.8709492377809208, 0.8709492377809208, 0.8709492377809208]
INFO : MRR score 0.8709492377809208
INFO : All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. [0.8217821782178217, 0.3052805280528053, 0.18712871287128713, 0.09504950495049502, 0.047772277227722766, 0.019108910891089105, 0.0009554455445544558]
